# 2MRS Group Catalogs

In this notebook we will explore the various 2MRS group catalogs and look at the most massive halos. We'll then compile the group catalogs to be used in the data analysis.

In [1]:
import os, sys
import math
#sys.path.append("/zfs/nrodd/NPTF-ID-Catalog/2MASS/")
# sys.path.append("/group/hepheno/smsharma/Fermi-LSS/2MASS/")


import numpy as np
import pandas as pd
import healpy as hp
import matplotlib.pyplot as plt
from astropy.cosmology import Planck13, z_at_value
from scipy.interpolate import UnivariateSpline, interp1d
from scipy.interpolate import InterpolatedUnivariateSpline
from astropy.io import fits
from tqdm import *
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance
from astropy.cosmology import Planck15, z_at_value
from scipy.optimize import fsolve

from units import *
# import tools
# from tools import lb2pix
import halo as hl
import Jpdf
import JBpdf
import Dpdf

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Huchra et al catalog (original 2MRS)

Huchra (original 2MASS) catalog. We don't actually do anything with it, just look at the 2MASS IDs etc to confirm the format.

In [2]:
# fits.open("/group/hepheno/smsharma/Fermi-LSS/AdditionalData/catalog/2mrs_1175_done.fits")[1].data['TMID']

## Cosmicflow

Cosmicflow catalog. Has a bunch of info that we use like common cluster names.

In [3]:
from astropy.io import ascii
cf_df = ascii.read("../DataFiles/Misc/ajaa259at2_mrt.txt").to_pandas()
cf_df.columns.values

array(['Nest', 'o_<DM>-gp', '<DM>-gp', 'e_<DM>-gp', '<Dist>-gp', 'Abell',
       'GName', 'Npv', 'PGC1', 'GLON-gp', 'GLAT-gp', 'SGLON-gp',
       'SGLAT-gp', 'logLKs', 'cf', 'Vsigma-proj', 'R2t-proj',
       '<Vhelio>-gp', '<Vgsr>-gp', '<Vls>-gp', '<Vcmb>-g', '<Vcmba>-gp',
       'Vrms-gp', 'Mvir', 'Mlum', 'LDC', 'HDC', '2M++', 'MK2011', 'Icnt'], dtype=object)

## Tully 3500

In [5]:
tully_3500 = ascii.read("../DataFiles/Misc/apjaa76dbt2_mrt.txt").to_pandas()

In [6]:
tully_3500.head()

,PGC1,PGC1+,Mem,GLON,GLAT,SGL,SGB,Ksmag,logK,Vh,VLS,ND,D,e_D,sigmaL,sigmaV,R2t,Rg,logMK,logMd
0,43296.0,43296.0,191.0,302.2241,21.6465,156.3251,-11.5819,4.61,12.71,3407.0,3142.0,59.0,36.96,3.0,595.0,800.0,1.612,0.893,14.624,14.717
1,46618.0,43296.0,30.0,307.8738,19.2865,159.6426,-6.8008,6.07,12.11,3340.0,3086.0,5.0,37.57,7.0,350.0,307.0,0.950,0.745,13.936,13.808
2,45174.0,43296.0,34.0,306.0424,32.5707,146.1941,-6.0422,6.42,11.96,3292.0,3059.0,4.0,44.03,8.0,307.0,300.0,0.833,0.693,13.764,13.756
3,40498.0,43296.0,24.0,297.5648,23.0823,153.9019,-15.4648,6.48,11.92,3258.0,2986.0,4.0,28.45,6.0,296.0,170.0,0.801,0.607,13.713,13.207
4,43557.0,43296.0,22.0,302.9333,36.4075,141.8602,-7.7062,6.82,11.81,3296.0,3066.0,7.0,39.89,8.0,267.0,163.0,0.724,0.428,13.582,13.017


In [7]:
tully_gal_3500 = ascii.read("../DataFiles/Misc/apjaa76dbt3_mrt.txt").to_pandas()

In [8]:
tully_gal_3500.head()

,PGC,Name,RAdeg,DEdeg,GLON,GLAT,SGL,SGB,T,Bmag,Ksmag,logK,Vh,VLS,D,e_D,PGC1
0,13418.0,NGC1399,54.6212,-35.4506,236.7163,-53.6356,262.5460,-42.0776,-4.6,10.35,6.44,11.25,1425.0,1283.0,22.08,11.0,13418.0
1,13179.0,NGC1365,53.4016,-36.1405,237.9564,-54.5979,261.8963,-40.9741,3.2,9.83,6.59,11.19,1664.0,1523.0,16.98,8.0,13418.0
2,13433.0,NGC1404,54.7163,-35.5942,236.9551,-53.5547,262.3357,-42.1253,-4.8,10.81,6.90,11.07,1873.0,1730.0,18.79,10.0,13418.0
3,13318.0,NGC1380,54.1149,-34.9761,235.9261,-54.0585,263.2770,-41.7608,-2.3,10.79,6.96,11.05,1855.0,1716.0,18.62,7.0,13418.0
4,13344.0,NGC1387,54.2377,-35.5066,236.8246,-53.9462,262.5532,-41.7605,-2.9,11.70,7.52,10.82,1265.0,1124.0,19.14,13.0,13418.0


In [11]:
# Get cosmological redshifts
z_cf = [z_at_value(Planck15.angular_diameter_distance, d*u.Mpc, zmax = 0.03) if not math.isnan(d) else np.nan for d in tqdm_notebook(tully_3500['D'].values) ]

In [12]:
celery = 299792. # Speed of light

In [13]:
twomrs_g_clusters_masses = []
twomrs_g_clusters_redshifts = []
twomrs_g_clusters_Glon = []
twomrs_g_clusters_Glat = []
twomrs_g_clusters_name = []
twomrs_g_clusters_rvir = []
twomrs_g_clusters_nmem = []
twomrs_g_clusters_twomasxj = []
twomrs_g_clusters_bibz = []
cf_GName = []

for iPGC1, PGC1 in enumerate(tqdm_notebook(tully_3500.PGC1.values)):
    twomrs_g_clusters_masses.append(10**tully_3500.logMK.values[iPGC1])
    twomrs_g_clusters_nmem.append(tully_3500.Mem.values[iPGC1])
    if not np.isnan(z_cf[iPGC1]):
        z_temp = z_cf[iPGC1]
    else:
        z_temp = tully_3500.VLS.values[iPGC1]/celery
    twomrs_g_clusters_redshifts.append(z_temp)
    twomrs_g_clusters_Glon.append(tully_3500.GLON.values[iPGC1])
    twomrs_g_clusters_Glat.append(tully_3500.GLAT.values[iPGC1])
    twomrs_g_clusters_rvir.append(tully_3500.R2t.values[iPGC1])
    
    twomrs_g_clusters_twomasxj.append(np.nan)
    twomrs_g_clusters_bibz.append(np.nan)
    
    twomrs_g_clusters_name.append(tully_gal_3500[tully_gal_3500.PGC == PGC1]['Name'].values[0])

    PGC1_loc_cf = cf_df['PGC1'] == PGC1
    GName = cf_df[PGC1_loc_cf]['GName'].values

    if len(GName) != 0:
        cf_GName.append(cf_df[PGC1_loc_cf]['GName'].values[0])
    else:
        cf_GName.append(np.nan)

## Tully catalog

Do some data munging to match Tully 2MRS and Tully clusters.

In [14]:
# Tully version of 2MRS table
twomrs_df =  pd.read_csv("../DataFiles/Misc/2MRS1175.csv",delimiter="|")
twomrs_df.head()

,pgc,ID_2MASXJ,RAJ,DEJ,Glon,Glat,SGL,SGB,K_c,H_c,...,r_ext,b/a,Flgs,Type,So,Vhel,e_V,C,Bib_z,Name
0,1,23595879+0042066,359.99500,0.70179,96.87975,-59.54168,293.35040,13.42997,11.503,11.882,...,1.473,0.72,0000,98,NN,24478,32,F,20112MRS.FLWO.0000H,23595879+0042066
1,2,00000168+4716282,0.00695,47.27456,113.95532,-14.69903,341.64417,20.73886,9.683,9.969,...,2.001,0.60,0333,3B_T,ZC,5017,9,N,1991RC3.9.C...0000d,UGC_12889
2,5,00000914+3244182,0.03814,32.73844,110.62066,-28.90421,326.17715,19.78053,10.608,10.912,...,1.461,0.80,0000,-4,ZC,10372,19,N,1999PASP..111..438F,IC_5370
3,12,00000865-0622263,0.03601,-6.37399,90.19206,-65.93002,286.42487,11.35101,11.224,11.497,...,1.609,0.26,0000,4,ZC,6531,45,6,20096dF...C...0000J,g0000086-062226
4,14,00000701+0816448,0.02926,8.27913,101.78542,-52.47268,300.91431,15.36726,10.758,11.066,...,1.602,0.60,0666,-5,ZC,11602,0,N,2000ApJS..129..547B,UGC_12890


In [15]:
# North clusters
twomrs_ng_clusters_df = pd.read_csv("../DataFiles/Misc//2MRS1175NG.csv",delimiter="|")

In [16]:
# South clusters
twomrs_sg_clusters_df = pd.read_csv("../DataFiles/Misc//2MRS1175SG.csv",delimiter="|")

In [17]:
# Combine north and south cluster catalogs
twomrs_g_clusters_df = pd.concat([twomrs_ng_clusters_df, twomrs_sg_clusters_df])

In [20]:
# Get unique galaxy groups
twomrs_g_clusters_Nests = pd.Series.unique(twomrs_g_clusters_df['Nest'])
twomrs_g_clusters_PGC1 = []

# Loop over unique galaxy groups and get the brightest central
for Nest in tqdm(twomrs_g_clusters_Nests):
    twomrs_g_clusters_PGC1.append(twomrs_g_clusters_df[twomrs_g_clusters_df['Nest'] == Nest]['PGC1'].values[0])

100%|██████████| 25486/25486 [00:17<00:00, 1444.30it/s]


In [21]:
# Conversion of Vmod into CMB-frame redshift, using eq. 14 of https://arxiv.org/pdf/1307.7213.pdf
# NO LONGER USED

omega_m = 0.27
omega_lambda = 0.73
q0 = 0.5*(omega_m - 2*omega_lambda)
c = 299792.
def Vmod(z, v):
    return c*z*(1+0.5*(1-q0)*z-(1/6.)*(1-q0-3*q0**2+1)*z**2) - v 

In [22]:
twomrs_g_clusters_df.columns.values

array(['pgc', 'GLong', 'GLat', 'SGL', 'SGB', 'Ty', 'Vhel', 'Vls', 'Vmod',
       'J-H', 'J-K', 'K_t', 'lgL_K', 'lgden', 'Nest', 'Ng', 'PGC1', 'D_v',
       'DM', 'SGLg', 'SGBg', 'lgLgp', 'cf', 'sigp', 'R2t', 'Vgp', 'Vbw',
       'eV', 'Sbw', 'rmsV', 'Rbw', 'eR', 'bwMass12', 'L_Mass12', 'LDC',
       'HDC', '2M++', 'SGX', 'SGY', 'SGZ'], dtype=object)

In [23]:
# Get relevant cluster property arrays and also cross-match with Tully 2MRS to get names of brightets clusters (this is what is most slow)

for PGC1 in tqdm(twomrs_g_clusters_PGC1):
    
    if PGC1 in tully_3500.PGC1.values: continue # If already in 3500 catalog, ignore

    PGC1_loc = twomrs_g_clusters_df['pgc'] == PGC1
    PGC1_loc_cf = cf_df['PGC1'] == PGC1
    if np.sum(PGC1_loc) == 0: continue
    vgp =  twomrs_g_clusters_df[PGC1_loc]['Vgp'].values[0]
    if vgp <= 3000: continue # Don't trust Tully 2MRS distances in this regime!
        
    twomrs_g_clusters_nmem.append(np.sum(twomrs_g_clusters_df['PGC1'] == PGC1))

    twomrs_g_clusters_masses.append(twomrs_g_clusters_df[PGC1_loc]['L_Mass12'].values[0]*10**12)    
    twomrs_g_clusters_redshifts.append(vgp/celery)
    twomrs_g_clusters_Glon.append(twomrs_g_clusters_df[PGC1_loc]['GLong'].values[0])
    twomrs_g_clusters_Glat.append(twomrs_g_clusters_df[PGC1_loc]['GLat'].values[0])
    twomrs_g_clusters_rvir.append(twomrs_g_clusters_df[PGC1_loc]['R2t'].values[0])
    twomrs_g_clusters_name.append(twomrs_df[twomrs_df['pgc'] == PGC1]['Name'].values[0])
    twomrs_g_clusters_twomasxj.append(twomrs_df[twomrs_df['pgc'] == PGC1]['ID_2MASXJ'].values[0])
    twomrs_g_clusters_bibz.append(twomrs_df[twomrs_df['pgc'] == PGC1]['Bib_z'].values[0])
    GName = cf_df[PGC1_loc_cf]['GName'].values

    if len(GName) != 0:
        cf_GName.append(cf_df[PGC1_loc_cf]['GName'].values[0])
    else:
        cf_GName.append(np.nan)

100%|██████████| 25486/25486 [02:48<00:00, 151.35it/s]


Now loop over halos to get J-factors, cvirs etc from `halo.py`.

In [24]:
# hm_bartels = hl.HaloModel(boost_model='bartels', alpha='alpha20', concentration_model='correa_Planck15', omega_m=Planck15.Om0, omega_lambda=Planck15.Ode0, h = Planck15.h)

hm_bartels = hl.HaloModel(boost_model='bartels', concentration_model='correa_Planck15', omega_m=Planck15.Om0, omega_lambda=Planck15.Ode0, h = Planck15.h)

# hm_bartels = hl.HaloModel(boost_model='sanchez-conde_tidal', alpha='alpha19', concentration_model='correa_Planck15', omega_m=Planck15.Om0, omega_lambda=Planck15.Ode0, h = Planck15.h)

Loading bartels
Interpolating concentration for DarkSky
Using model correa_Planck15


In [25]:
mulog10Jfnb_arr = []
sigmalog10Jfnb_arr = []
mulog10Jf_arr = []
sigmalog10Jf_arr = []

mulog10JBfnb_arr = []
sigmalog10JBfnb_arr = []
mulog10JBf_arr = []
sigmalog10JBf_arr = []

mulog10Df_arr = []
sigmalog10Df_arr = []

twomrs_g_clusters_rvir_der = []
twomrs_g_clusters_cvir_der = []

twomrs_g_clusters_sigmacvir_der = []

twomrs_g_clusters_sigmalogmasses = []

test = []

# Interpolated cvir errors
cvir_err_arys = np.load("/tigress/nrodd/DM-Catalog-Scan/DataFiles/Misc/cvir_err.npy")
cvir_err_inter = InterpolatedUnivariateSpline(cvir_err_arys[0],cvir_err_arys[1])

Ji = Jpdf.Jc(hm_bartels)
JBi = JBpdf.Jc(hm_bartels)
Di = Dpdf.Dc(hm_bartels)


for i in tqdm_notebook(range(len(twomrs_g_clusters_masses))):
    hm_bartels.cvir = -999
    hm_bartels.rvir = -999
        
    mvir = twomrs_g_clusters_masses[i]*M_s # This is in keV...

    # Some clusters have a 0 mass, if so set mvir to be tiny
    if twomrs_g_clusters_masses[i] == 0.:
        mvir = 0.01*M_s
    
    z = twomrs_g_clusters_redshifts[i]
        
    if 0 < z < 0.03 and np.log10(mvir/M_s) > 10.1: 
        
        cvir = hm_bartels.c_vir(mvir, z)
        rvir = hm_bartels.r_vir(mvir, z)

        mvir /= M_s # Msun

        mulog10M = np.log10(mvir)
        sigmalog10M = mulog10M*0.01
        
        mulog10c = np.log10(cvir)
        #sigmalog10c = 0.15 # NB: Might want to update this later!    
        sigmalog10c = cvir_err_inter(np.log10(mvir))
        
        

        mulog10Jnb, sigmalog10Jnb, mulog10J, sigmalog10J = Ji.log10Jmusig(mulog10M, sigmalog10M, mulog10c, sigmalog10c, z)
        mulog10JBnb, sigmalog10JBnb, mulog10JB, sigmalog10JB = JBi.log10Jmusig(mulog10M, sigmalog10M, mulog10c, sigmalog10c, z)
        mulog10D, sigmalog10D = Di.log10Dmusig(mulog10M, sigmalog10M, z)
    
    else:
        mulog10Jnb, sigmalog10Jnb, mulog10J, sigmalog10J = -999., -999., -999., -999.
        mulog10JBnb, sigmalog10JBnb, mulog10JB, sigmalog10JB = -999., -999., -999., -999.
        mulog10D, sigmalog10D = -999., -999.
        cvir, rvir = -999, -999
        sigmalog10c = -999
        sigmalog10M = -999
        
    twomrs_g_clusters_sigmacvir_der.append(sigmalog10c)
        
    mulog10Jfnb_arr.append(mulog10Jnb)
    sigmalog10Jfnb_arr.append(sigmalog10Jnb)
    mulog10Jf_arr.append(mulog10J)
    sigmalog10Jf_arr.append(sigmalog10J)
    
    mulog10JBfnb_arr.append(mulog10JBnb)
    sigmalog10JBfnb_arr.append(sigmalog10JBnb)
    mulog10JBf_arr.append(mulog10JB)
    sigmalog10JBf_arr.append(sigmalog10JB)
    
    mulog10Df_arr.append(mulog10D)
    sigmalog10Df_arr.append(sigmalog10D)
    
    twomrs_g_clusters_rvir_der.append(rvir)
    twomrs_g_clusters_cvir_der.append(cvir)
    
    twomrs_g_clusters_sigmalogmasses.append(sigmalog10M)

In [26]:
print np.min(mulog10Jf_arr), np.max(mulog10Jf_arr)

-999.0 23.2867225367


Make dataframe and add in information we just inferred

In [27]:
nhalos = len(mulog10Jf_arr)

sales = zip(twomrs_g_clusters_name,
            mulog10Jf_arr,
            sigmalog10Jf_arr,
            mulog10Jfnb_arr,
            sigmalog10Jfnb_arr,
            mulog10JBf_arr,
            sigmalog10JBf_arr,
            mulog10JBfnb_arr,
            sigmalog10JBfnb_arr,
            mulog10Df_arr,
            sigmalog10Df_arr,
            np.log10(twomrs_g_clusters_masses),
            twomrs_g_clusters_sigmalogmasses,
            twomrs_g_clusters_redshifts,
            twomrs_g_clusters_Nests,
            twomrs_g_clusters_nmem,
            twomrs_g_clusters_Glon,
            twomrs_g_clusters_Glat,
            twomrs_g_clusters_cvir_der,
            twomrs_g_clusters_sigmacvir_der,
            np.array(twomrs_g_clusters_rvir_der)/(.001*Mpc),
            np.array(twomrs_g_clusters_rvir_der)/(.001*Mpc)/np.array(twomrs_g_clusters_cvir_der),
            twomrs_g_clusters_twomasxj,
            twomrs_g_clusters_bibz,
            cf_GName)
labels = ["Name", "mulog10J","siglog10J","mulog10Jnb","siglog10Jnb",
          "mulog10JB","siglog10JB","mulog10JBnb","siglog10JBnb",
          "mulog10D","siglog10D",
          "log10Mvir","sigmalog10Mvir", "z", "Nest", "Ng", "l", "b", "cvir", "sigmalog10cvir","rvir", "rs", "ID_2MASXJ", "Bib_z", "GName"]
df = pd.DataFrame.from_records(sales, columns=labels)
df = df[df["z"] < 0.03] # Cut to avoid the issue with crazy massive halos from overincluding satellite contribution
df = df[df["mulog10J"] != -999.] 
df = df[df["Name"] != "Milky Way"] # Remove Milky Way
dftop = df.sort_values(["mulog10J"],ascending=False)[:nhalos]

Add 3FGL-related information

In [28]:
# Load in the 3FGL catalog 
source_3fg_df = pd.read_csv('../DataFiles/Catalogs//3fgl.dat', sep='|', comment='#')

In [29]:
# Remove whitespace from column names
source_3fg_df.rename(columns=lambda x: x.strip(), inplace=True)
for col in source_3fg_df.columns.values:
    try:
        source_3fg_df[col] = source_3fg_df[
            col].map(str.strip)
    except TypeError:
        continue

# Convert to numeric data
source_3fg_df = source_3fg_df.convert_objects(
    convert_numeric=True)

/tigress/smsharma/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  if sys.path[0] == '':


In [35]:
dftop[dftop['GName'] == 'Virgo']

,Name,mulog10J,siglog10J,mulog10Jnb,siglog10Jnb,mulog10JB,siglog10JB,mulog10JBnb,siglog10JBnb,mulog10D,...,Ng,l,b,cvir,sigmalog10cvir,rvir,rs,ID_2MASXJ,Bib_z,GName
210,NGC4472,19.115659,0.352758,18.372857,0.338266,18.864802,0.381959,18.121999,0.368616,20.329794,...,714.0,283.9352,74.5198,6.363708,0.180669581405,2023.322175,317.947047,NaN,NaN,Virgo


In [33]:
dftop[dftop['Name'] == 'NGC1316']

,Name,mulog10J,siglog10J,mulog10Jnb,siglog10Jnb,mulog10JB,siglog10JB,mulog10JBnb,siglog10JBnb,mulog10D,...,Ng,l,b,cvir,sigmalog10cvir,rvir,rs,ID_2MASXJ,Bib_z,GName
907,NGC1316,18.017668,0.361676,17.364965,0.351015,17.789404,0.390367,17.136701,0.38051,19.104108,...,13.0,239.9775,-56.6784,8.465389,0.177784753047,873.627337,103.19991,NaN,NaN,Fornax


In [34]:
dftop[dftop['Name'] == 'NGC1399']

,Name,mulog10J,siglog10J,mulog10Jnb,siglog10Jnb,mulog10JB,siglog10JB,mulog10JBnb,siglog10JBnb,mulog10D,...,Ng,l,b,cvir,sigmalog10cvir,rvir,rs,ID_2MASXJ,Bib_z,GName
906,NGC1399,18.310999,0.370018,17.623174,0.347762,18.076683,0.398209,17.388858,0.377617,19.423029,...,120.0,236.6231,-53.8838,7.834312,0.178526982061,1105.77443,141.145051,NaN,NaN,NaN


In [36]:
# # Change Virgo and Fornax centers
# # DON'T DO THIS ANYMORE WITH T17!

# dftop.loc[dftop['GName'] == 'Virgo','l'] = 283.7777978
# dftop.loc[dftop['GName'] == 'Virgo','b'] = 74.4911308

# dftop.loc[dftop['GName'] == 'Fornax','l'] = 236.72
# dftop.loc[dftop['GName'] == 'Fornax','b'] = -53.64

In [37]:
# Astropy-formatted coordinates of cluster and 3FGL
c2 = SkyCoord("galactic", l=dftop['l']*u.deg, b=dftop['b']*u.deg)
c3 = SkyCoord("galactic", l=source_3fg_df['_Lii']*u.deg, b=source_3fg_df['_Bii']*u.deg)

In [38]:
# Select 3FGL sources within some radius

threefgl0p5deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, .5*u.deg)
for i in range(len(dftop)):
    threefgl0p5deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

threefgl1deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 1*u.deg)
for i in range(len(dftop)):
    threefgl1deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

threefgl2deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 2*u.deg)
for i in range(len(dftop)):
    threefgl2deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)
    
threefgl5deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 5*u.deg)
for i in range(len(dftop)):
    threefgl5deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)
    
threefgl10deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 10*u.deg)
for i in range(len(dftop)):
    threefgl10deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

In [83]:
# 3FGL sources within x degrees
dftop['3FGL0.5'] = pd.Series(threefgl0p5deg, index=dftop.index)
dftop['3FGL1'] = pd.Series(threefgl1deg, index=dftop.index)
dftop['3FGL2'] = pd.Series(threefgl2deg, index=dftop.index)
dftop['3FGL5'] = pd.Series(threefgl5deg, index=dftop.index)
dftop['3FGL10'] = pd.Series(threefgl10deg, index=dftop.index)

In [84]:
# Number of 3FGL sources within 5, 10 degrees
dftop['N3FGL5'] = pd.Series([len(x) for x in threefgl5deg], index=dftop.index)
dftop['N3FGL10'] = pd.Series([len(x) for x in threefgl10deg], index=dftop.index)

In [50]:
# Angular extension

# from astropy.cosmology import FlatLambdaCDM
# cosmo = FlatLambdaCDM(H0=hm_bartels.h*100, Om0=hm_bartels.omega_m)
cosmo = Planck15

theta_s = np.arctan(dftop['rs'].values/(cosmo.angular_diameter_distance(dftop['z']).value*1000))*(180/np.pi)
theta_vir = np.arctan(dftop['rvir'].values/(cosmo.angular_diameter_distance(dftop['z']).value*1000))*(180/np.pi)

dftop['theta_s'] = pd.Series(theta_s, index=dftop.index)
dftop['theta_vir'] = pd.Series(theta_vir, index=dftop.index)

In [52]:
dftop['ra'] = c2.transform_to('icrs').ra.degree  
dftop['dec'] = c2.transform_to('icrs').dec.degree  

In [53]:
dftop['dA'] = Planck15.angular_diameter_distance(dftop['z'])

In [131]:
dftop['log10M200'] = [hm_bartels.M200_cvir((10**log10Mvir)*M_s, cvir, z)/M_s for log10Mvir, cvir, z in tqdm_notebook(zip(dftop.log10Mvir, dftop.cvir,dftop.z))]

In [137]:
dftop['log10M200'] = np.log10(dftop['log10M200'])

In [134]:
dftop['r200'] = [hm_bartels.c200_from_cvir(cvir,z)/cvir*rvir for cvir, rvir, z in tqdm_notebook(zip(dftop.cvir, dftop.rvir, dftop.z))]

In [139]:
dftop.to_csv("../DataFiles/Catalogs/2MRSLocalTully_ALL_DATAPAPER_Planck15_extra2_v8.csv")

In [2]:
dftop = pd.read_csv("../DataFiles/Catalogs/2MRSLocalTully_ALL_DATAPAPER_Planck15_extra2_v8.csv")

/tigress/smsharma/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## For catalog

In [3]:
dfcat = dftop[['Name',"GName","mulog10J","siglog10J","mulog10Jnb","siglog10Jnb",
          "mulog10JB","siglog10JB","mulog10JBnb","siglog10JBnb",
          "mulog10D","siglog10D",'log10Mvir','sigmalog10Mvir','cvir','sigmalog10cvir','z','dA','rvir','rs',"Ng", "l", "b",'ra','dec',
      'theta_s','log10M200','r200']].reset_index(drop=True)
dfcat

,Name,GName,mulog10J,siglog10J,mulog10Jnb,siglog10Jnb,mulog10JB,siglog10JB,mulog10JBnb,siglog10JBnb,...,rvir,rs,Ng,l,b,ra,dec,theta_s,log10M200,r200
0,Andromeda,NaN,19.793990,0.367214,19.233062,0.360742,19.585515,0.394558,19.024586,0.388542,...,369.956241,33.614916,38.0,121.5054,-21.7886,11.102555,41.063681,2.566277,12.337597,280.878456
1,NGC4472,Virgo,19.115659,0.352758,18.372857,0.338266,18.864802,0.381959,18.121999,0.368616,...,2023.322175,317.947047,714.0,283.9352,74.5198,187.755776,12.405943,1.155746,14.551535,1513.597395
2,NGC5128,NaN,18.896069,0.378974,18.279186,0.356886,18.679241,0.406024,18.062358,0.385489,...,546.684614,55.601881,30.0,307.8845,17.0814,199.624611,-45.529967,0.877548,12.846536,413.974493
3,NGC0253,NaN,18.761680,0.372880,18.170785,0.358277,18.547804,0.400180,17.956908,0.386610,...,477.977460,46.732498,4.0,98.2371,-87.8903,11.885476,-25.208151,0.767167,12.671602,362.310511
4,Maffei 1,NaN,18.688100,0.370669,18.106905,0.358768,18.475447,0.398041,17.894252,0.386983,...,447.790545,43.065708,8.0,136.2315,-0.4405,39.905069,59.608401,0.713108,12.586629,339.569211
5,NGC 6822,NaN,18.594351,0.375988,18.163484,0.368982,18.408005,0.400740,17.977139,0.394174,...,97.909433,6.472403,1.0,25.3411,-18.3951,296.237312,-14.800367,0.772532,10.605891,74.851129
6,NGC3031,NaN,18.588134,0.369610,18.012834,0.359326,18.376581,0.396991,17.801281,0.387434,...,427.631440,40.519518,41.0,141.8791,40.8662,149.071762,69.216095,0.636027,12.526661,324.407823
7,NGC4696,Centaurus,18.346298,0.351851,17.606330,0.338428,18.095882,0.381128,17.355914,0.368773,...,1971.531373,308.105856,191.0,302.2241,21.6465,191.984798,-41.220439,0.477618,14.520507,1477.817544
8,NGC1399,NaN,18.310999,0.370018,17.623174,0.347762,18.076683,0.398209,17.388858,0.377617,...,1105.774430,141.145051,120.0,236.6231,-53.8838,54.318357,-35.389692,0.447034,13.765579,832.990558
9,IC0356,NaN,18.271854,0.361928,17.618154,0.350952,18.043422,0.390607,17.389723,0.380459,...,881.965258,104.410002,15.0,138.0615,12.6955,60.297553,69.775669,0.431612,13.470629,665.641536


In [4]:
dfcat.to_csv("/tigress/smsharma/DMCat/DMCat/DMCat.csv")

## For paper

In [5]:
catalog = pd.read_csv("../DataFiles/Catalogs/2MRSLocalTully_ALL_DATAPAPER_Planck15_extra2_v8.csv")

In [6]:
catalog['zred'] = catalog['z']*1000
catalog['log10cvir'] = np.log10(catalog['cvir'])
catalog['theta_s'] = catalog['theta_s']
catalog['bsh'] = 10**catalog['mulog10J']/10**catalog['mulog10Jnb'] - 1
catalog['paperName'] = [catalog['Name'][i] + '/' + catalog['GName'][i] if isinstance(catalog['GName'][i], str) else catalog['Name'][i] for i in range(len(catalog))]

In [7]:
catalog['log10Jerr'] = [str(catalog['mulog10J'].values[i]) [:5] +str('$\\pm$') + str(catalog['siglog10J'].values[i]) [:4] for i in range(len(catalog))]#[:4]

In [8]:
catalog['log10cerr'] = [str(catalog['log10cvir'].values[i]) [:4] +str('$\\pm$') + str(catalog['sigmalog10cvir'].values[i]) [:4] for i in range(len(catalog))]#[:4]

In [9]:
catalog['log10Merr'] = [str(catalog['log10Mvir'].values[i]) [:4] +str('$\\pm$') + str(catalog['sigmalog10Mvir'].values[i]) [:4] for i in range(len(catalog))]#[:4]

In [10]:
passed_halos = np.load("/tigress/nrodd/DM-Catalog-Scan/Scan-Small-ROI/notebooks/data_passed_halos.npy")

In [11]:
catalog['in'] = ['\checkmark' if i in passed_halos else '' for i in range(len(catalog))]#[:4]

In [12]:
catalog_paper = catalog.iloc[passed_halos][['paperName','log10Jerr','log10Merr','zred','l','b','log10cerr','theta_s','bsh']].round(2).to_latex(index = False)

In [13]:
print catalog_paper

\begin{tabular}{lllrrrlrr}
\toprule
                paperName &       log10Jerr &      log10Merr &   zred &       l &      b &      log10cerr &  theta\_s &   bsh \\
\midrule
            NGC4472/Virgo &  19.11\$\textbackslashpm\$0.35 &  14.6\$\textbackslashpm\$0.14 &   3.58 &  283.94 &  74.52 &  0.80\$\textbackslashpm\$0.18 &     1.16 &  4.53 \\
                  NGC0253 &  18.76\$\textbackslashpm\$0.37 &  12.7\$\textbackslashpm\$0.12 &   0.79 &   98.24 & -87.89 &  1.00\$\textbackslashpm\$0.17 &     0.77 &  2.90 \\
                  NGC3031 &  18.58\$\textbackslashpm\$0.36 &  12.6\$\textbackslashpm\$0.12 &   0.83 &  141.88 &  40.87 &  1.02\$\textbackslashpm\$0.17 &     0.64 &  2.76 \\
        NGC4696/Centaurus &  18.34\$\textbackslashpm\$0.35 &  14.6\$\textbackslashpm\$0.14 &   8.44 &  302.22 &  21.65 &  0.80\$\textbackslashpm\$0.18 &     0.48 &  4.50 \\
                  NGC1399 &  18.31\$\textbackslashpm\$0.37 &  13.8\$\textbackslashpm\$0.13 &   4.11 &  236.62 & -53.88 &  0.89\$\textb

In [14]:
catalog_app = catalog[:30][['paperName','log10Jerr','log10Merr','zred','l','b','log10cerr','theta_s','bsh']]

In [15]:
catalog_app['maxTS'] = [np.load("/tigress/nrodd/DM-Catalog-Scan/Scan-Small-ROI//data/Tully/LL2_TSmx_lim_b_emin4_emax30_o" + str(iobj) + "_data.npz")['TSmx'][0] for iobj in tqdm_notebook(range(len(catalog_app)))]

In [16]:
catalog_app['in'] = catalog['in']

In [17]:
print catalog_app.round(2).to_latex(index = False)

\begin{tabular}{lllrrrlrrrl}
\toprule
             paperName &       log10Jerr &      log10Merr &   zred &       l &      b &      log10cerr &  theta\_s &   bsh &  maxTS &          in \\
\midrule
             Andromeda &  19.79\$\textbackslashpm\$0.36 &  12.4\$\textbackslashpm\$0.12 &   0.17 &  121.51 & -21.79 &  1.04\$\textbackslashpm\$0.17 &     2.57 &  2.64 &   2.92 &             \\
         NGC4472/Virgo &  19.11\$\textbackslashpm\$0.35 &  14.6\$\textbackslashpm\$0.14 &   3.58 &  283.94 &  74.52 &  0.80\$\textbackslashpm\$0.18 &     1.16 &  4.53 &   1.04 &  \textbackslashcheckmark \\
               NGC5128 &  18.89\$\textbackslashpm\$0.37 &  12.9\$\textbackslashpm\$0.12 &   0.82 &  307.88 &  17.08 &  0.99\$\textbackslashpm\$0.17 &     0.88 &  3.14 &   0.00 &             \\
               NGC0253 &  18.76\$\textbackslashpm\$0.37 &  12.7\$\textbackslashpm\$0.12 &   0.79 &   98.24 & -87.89 &  1.00\$\textbackslashpm\$0.17 &     0.77 &  2.90 &   0.63 &  \textbackslashcheckmark \\
      

In [148]:
Planck15.Om(0), Planck15.h

(0.3075, 0.6774)

In [146]:
Planck15.__doc__

u'Planck15 instance of FlatLambdaCDM cosmology\n\n(from Planck Collaboration 2016, A&A, 594, A13 (Paper XIII), Table 4 (TT, TE, EE + lowP + lensing + ext))'

In [151]:
cat_ds = pd.read_csv("../DataFiles/Catalogs/DarkSky_ALL_200,200,200_v3.csv")

In [188]:
cat_ds

,Unnamed: 0,mulog10J_true,mulog10J_inf,siglog10J_inf,mulog10Jnb_inf,siglog10Jnb_inf,l,b,rs,rvir_true,...,logMvir_inf,logMvir+_inf,theta_vir,lumi,masked,N3FGL 5,3FGL 0.5,3FGL 1,3FGL 2,3FGL 5
0,14265,18.642517,18.700522,0.352574,17.961355,0.344853,-64.704712,6.791400,201.514532,1877.428810,...,14.614549,14.710479,3.124791,3.942819e+11,-999.0,4,[],[],[],['3FGL J1223.0-5622' '3FGL J1128.6-5434' '3FGL...
1,6214,18.203234,18.251704,0.205549,17.450622,0.205547,-136.568686,-70.197089,699.106083,3593.880942,...,15.447989,15.445783,2.081835,2.207309e+12,-999.0,5,[],[],[],['3FGL J0238.4-3117' '3FGL J0228.7-3106' '3FGL...
2,16368,18.264156,18.162999,0.411271,17.512531,0.387320,16.417404,-30.091858,99.954937,915.909878,...,13.533921,13.751435,3.223154,5.863489e+10,-999.0,4,[],[],[],['3FGL J2025.1-2858' '3FGL J2036.8-2830' '3FGL...
3,16525,17.927609,18.018616,0.457017,17.483412,0.415975,23.076352,72.321453,20.271802,238.130814,...,11.951959,12.156428,3.968874,5.123379e+09,-999.0,2,[],[],[],['3FGL J1427.0+2347' '3FGL J1417.8+2540']
4,25189,18.072250,17.981714,0.312191,17.226584,0.303714,46.336464,-3.681863,300.864818,2359.472408,...,14.774726,14.846399,1.614034,5.568748e+11,-999.0,10,[],['3FGL J1931.1+0937'],['3FGL J1931.1+0937'],['3FGL J1933.4+0727' '3FGL J1910.9+0906' '3FGL...
5,14266,18.022771,17.955857,0.300253,17.235973,0.294635,-79.286628,9.240707,227.954932,1612.912733,...,14.298163,14.424198,2.141879,2.151451e+11,-999.0,6,[],[],[],['3FGL J1033.4-5035' '3FGL J1048.8-5006' '3FGL...
6,16373,18.097282,17.951379,0.366097,17.317730,0.348137,115.401350,23.164206,121.061048,756.898215,...,13.239862,13.492216,4.195736,3.857239e+10,-999.0,2,[],[],[],['3FGL J2005.2+7752' '3FGL J1935.5+8355']
7,23802,17.920952,17.925141,0.395231,17.207807,0.390967,14.917346,-5.561823,129.607555,1458.648238,...,14.233532,14.368220,1.308508,1.893906e+11,-999.0,7,[],[],[],['3FGL J1833.6-2103' '3FGL J1819.5-2045c' '3FG...
8,16402,17.817139,17.924106,0.457922,17.350385,0.421593,-143.949297,43.169542,39.933141,389.424435,...,12.612564,12.891834,3.108032,1.588490e+10,-999.0,0,[],[],[],[]
9,16374,18.475933,17.917234,0.494641,17.335005,0.452072,-105.236380,-9.125932,58.004362,663.863382,...,12.699798,12.981219,4.036291,1.803245e+10,-999.0,4,[],[],['3FGL J0744.8-4028'],['3FGL J0718.7-4319' '3FGL J0744.8-4028' '3FGL...


In [ ]:
(10**cat_ds['logMvir+_inf'] - 10**cat_ds['logMvir_inf'])/10**cat_ds['logMvir_inf']

In [187]:
(10**cat_ds['logMvir+_inf'] - 10**cat_ds['logMvir_inf'])/10**cat_ds['logMvir_inf']

0       0.247184
1      -0.005065
2       0.650113
3       0.601287
4       0.179434
5       0.336704
6       0.787946
7       0.363603
8       0.902261
9       0.911707
10      0.816433
11      0.406531
12      0.298736
13      0.272890
14      0.797791
15      0.270747
16      0.231575
17      0.163797
18      0.261348
19      0.735818
20      0.281556
21      0.469633
22      0.168587
23      0.911681
24      0.251707
25      0.510320
26      0.782606
27      0.721713
28      0.804533
29      0.421171
          ...   
9970    0.906831
9971    0.747158
9972    0.609306
9973    0.906594
9974    0.806625
9975    0.771119
9976    0.691878
9977    0.773096
9978    0.802485
9979    0.835928
9980    0.855792
9981    0.851378
9982    0.864396
9983    0.774559
9984    0.842903
9985    0.803507
9986    0.875940
9987    0.828085
9988    0.665833
9989    0.701089
9990    0.737980
9991    0.617387
9992    0.677048
9993    0.727129
9994    0.557509
9995    0.541958
9996    0.891803
9997    0.8961

In [174]:
catalog.sigmalog10Mvir.values

array([ 0.12441,  0.14656,  0.1295 , ...,  0.10696,  0.10403,  0.1025 ])

In [182]:
(10**(catalog.log10Mvir.values) - 10**(catalog.log10Mvir - catalog.sigmalog10Mvir.values ))/10**(catalog.log10Mvir.values)

0        0.249086
1        0.286424
2        0.257836
3        0.254839
4        0.253379
5        0.218534
6        0.252347
7        0.285898
8        0.273376
9        0.268423
10       0.264894
11       0.293744
12       0.217670
13       0.293402
14       0.246627
15       0.259116
16       0.268221
17       0.245742
18       0.239078
19       0.295842
20       0.261739
21       0.282569
22       0.266281
23       0.257562
24       0.253293
25       0.234403
26       0.251055
27       0.263810
28       0.274796
29       0.244177
           ...   
16990    0.217922
16991    0.223556
16992    0.217021
16993    0.216895
16994    0.218156
16995    0.223824
16996    0.216949
16997    0.217832
16998    0.217994
16999    0.214855
17000    0.216733
17001    0.217201
17002    0.214909
17003    0.216895
17004    0.216534
17005    0.215903
17006    0.215108
17007    0.216787
17008    0.214584
17009    0.216336
17010    0.224700
17011    0.212229
17012    0.215848
17013    0.214529
17014    0

In [180]:
(10**(catalog.log10Mvir.values + catalog.sigmalog10Mvir.values) - 10**(catalog.log10Mvir))/10**(catalog.log10Mvir.values)

0        0.331711
1        0.401393
2        0.347411
3        0.341992
4        0.339368
5        0.279646
6        0.337519
7        0.400361
8        0.376227
9        0.366910
10       0.360348
11       0.415918
12       0.278233
13       0.415233
14       0.327364
15       0.349740
16       0.366533
17       0.325806
18       0.314196
19       0.420136
20       0.354534
21       0.393863
22       0.362919
23       0.346914
24       0.339214
25       0.306171
26       0.335211
27       0.358345
28       0.378923
29       0.323061
           ...   
16990    0.278645
16991    0.287923
16992    0.277174
16993    0.276968
16994    0.279028
16995    0.288368
16996    0.277056
16997    0.278498
16998    0.278763
16999    0.273650
17000    0.276703
17001    0.277468
17002    0.273738
17003    0.276968
17004    0.276380
17005    0.275352
17006    0.274060
17007    0.276792
17008    0.273210
17009    0.276057
17010    0.289823
17011    0.269404
17012    0.275264
17013    0.273122
17014    0

In [166]:
10**catalog.sigmalog10Mvir/10**catalog.log10Mvir

0        4.824030e-13
1        3.094283e-15
2        1.511820e-13
3        2.252942e-13
4        2.734639e-13
5        2.500864e-11
6        3.135451e-13
7        3.328434e-15
8        1.860758e-14
9        3.645356e-14
10       5.870161e-14
11       1.114896e-15
12       2.790037e-11
13       1.169614e-15
14       6.667914e-13
15       1.274236e-13
16       3.746451e-14
17       7.489969e-13
18       1.789205e-12
19       8.304889e-16
20       8.969950e-14
21       5.274970e-15
22       4.869339e-14
23       1.567978e-13
24       2.765986e-13
25       3.280953e-12
26       3.720060e-13
27       6.792193e-14
28       1.532992e-14
29       9.195598e-13
             ...     
16990    2.702402e-11
16991    1.320961e-11
16992    3.028656e-11
16993    3.077372e-11
16994    2.623494e-11
16995    1.276556e-11
16996    3.056399e-11
16997    2.733380e-11
16998    2.677873e-11
16999    3.981530e-11
17000    3.141160e-11
17001    2.960397e-11
17002    3.954395e-11
17003    3.077372e-11
17004    3

In [189]:
catalog

,Unnamed: 0,Name,mulog10J,siglog10J,mulog10Jnb,siglog10Jnb,mulog10JB,siglog10JB,mulog10JBnb,siglog10JBnb,...,N3FGL10,zred,log10cvir_inf,bsh,log10cvir,paperName,log10Jerr,log10cerr,log10Merr,in
0,3072,Andromeda,19.793990,0.367214,19.233062,0.360742,19.585515,0.394558,19.024586,0.388542,...,25,0.169502,1.041618,2.638552,1.041618,Andromeda,19.79$\pm$0.36,1.04$\pm$0.17,12.4$\pm$0.12,
1,210,NGC4472,19.115659,0.352758,18.372857,0.338266,18.864802,0.381959,18.121999,0.368616,...,26,3.576776,0.803710,4.530983,0.803710,Virgo,19.11$\pm$0.35,0.80$\pm$0.18,14.6$\pm$0.14,\checkmark
2,2565,NGC5128,18.896069,0.378974,18.279186,0.356886,18.679241,0.406024,18.062358,0.385489,...,31,0.821052,0.992647,3.138884,0.992647,NGC5128,18.89$\pm$0.37,0.99$\pm$0.17,12.9$\pm$0.12,
3,3211,NGC0253,18.761680,0.372880,18.170785,0.358277,18.547804,0.400180,17.956908,0.386610,...,22,0.789355,1.009788,2.898484,1.009788,NGC0253,18.76$\pm$0.37,1.00$\pm$0.17,12.7$\pm$0.12,\checkmark
4,3239,Maffei 1,18.688100,0.370669,18.106905,0.358768,18.475447,0.398041,17.894252,0.386983,...,39,0.782563,1.016943,2.812370,1.016943,Maffei 1,18.68$\pm$0.37,1.01$\pm$0.17,12.6$\pm$0.12,
5,3074,NGC 6822,18.594351,0.375988,18.163484,0.368982,18.408005,0.400740,17.977139,0.394174,...,27,0.108474,1.179759,1.696911,1.179759,NGC 6822,18.59$\pm$0.37,1.17$\pm$0.17,10.7$\pm$0.10,
6,2809,NGC3031,18.588134,0.369610,18.012834,0.359326,18.376581,0.396991,17.801281,0.387434,...,34,0.825580,1.023405,2.760971,1.023405,NGC3031,18.58$\pm$0.36,1.02$\pm$0.17,12.6$\pm$0.12,\checkmark
7,0,NGC4696,18.346298,0.351851,17.606330,0.338428,18.095882,0.381128,17.355914,0.368773,...,33,8.438346,0.806104,4.495003,0.806104,Centaurus,18.34$\pm$0.35,0.80$\pm$0.18,14.6$\pm$0.14,\checkmark
8,906,NGC1399,18.310999,0.370018,17.623174,0.347762,18.076683,0.398209,17.388858,0.377617,...,17,4.108259,0.894001,3.873318,0.894001,NGC1399,18.31$\pm$0.37,0.89$\pm$0.17,13.8$\pm$0.13,\checkmark
9,2153,IC0356,18.271854,0.361928,17.618154,0.350952,18.043422,0.390607,17.389723,0.380459,...,27,3.143894,0.926709,3.505046,0.926709,IC0356,18.27$\pm$0.36,0.92$\pm$0.17,13.5$\pm$0.13,
